In [40]:
import pandas as pd
import numpy as np
import os
import gzip
import io
from scripts.check_overlaps import check_overlaps, check_overlaps_and_annotate

Rheinbay 2020 Supplementary Table 5<br>
We filter on the basis of 'Final filter judgement' = TRUE<br>
Then we take the unique element IDs<br>

In [ ]:
df = pd.read_excel('data/Rheinbay 2020/41586_2020_1965_MOESM4_ESM.xlsx', engine='openpyxl', sheet_name='Table 5 Non-coding point mut dr')
# df1 = pd.read_excel('data/Rheinbay 2020/41586_2020_1965_MOESM4_ESM.xlsx', engine='openpyxl', sheet_name='Table 4 Protein-coding point mu')
df['file_name'] = df['ID'].apply(lambda x: x.split('::')[0]+'.bed')
df['file_name'] = df['file_name'].apply(lambda x: x.replace('mirna_', 'mirna.') if 'mirna_pre' in x or 'mirna_mat' in x else x)
noncoding_elements = list(df[df['Final filter judgement'] == True]['ID'].unique())

In [ ]:
file_df = pd.read_csv('data/PCAWG_mutations_to_elements.icgc.public.txt', sep='\t')
file_df

In [115]:
rheinbay_noncoding_drivers = file_df[file_df['reg_id'].isin(noncoding_elements)][['mut_chr', 'mut_pos1', 'mut_pos2', 'mut_ref', 'mut_alt', 'region_type']].drop_duplicates().reset_index(drop=True)
rheinbay_noncoding_drivers.rename({'mut_chr': 'chr', 'mut_pos1': 'start', 'mut_pos2': 'end', 'mut_ref': 'ref', 'mut_alt': 'alt'}, axis=1, inplace=True)
rheinbay_noncoding_drivers

,chr,start,end,ref,alt,region_type
0,chr11,111231125,111231125,G,C,enhancers.bed
1,chr11,111232885,111232885,C,T,enhancers.bed
2,chr7,86866372,86866372,T,C,enhancers.bed
3,chr1,2267139,2267139,G,A,enhancers.bed
4,chr11,111226848,111226848,T,C,enhancers.bed
...,...,...,...,...,...,...
895,chr11,65194294,65194295,-,CATAATATTTTAAAA,lncrna.ncrna.bed
896,chr11,65269272,65269273,AA,-,lncrna.ncrna.bed
897,chr11,65267659,65267659,A,-,lncrna.ncrna.bed
898,chr11,65209144,65209148,TTTAT,-,lncrna.ncrna.bed


In [42]:
elements_list = os.listdir('data/gene-coding-and-non-coding-elements/')
elements_list.remove('allCombined.zip')
elements_list.remove('mapFiles')

In [4]:
arr = []
df['chr'] = ''
df['start'] = 0
df['end'] = 0
for element in elements_list:
    print(element)
    tempdf = df[df['file_name'] == element]
    file_df = pd.read_csv('data/gene-coding-and-non-coding-elements/'+element, sep='\t', header=None)
    file_df.columns = ['chrom', 'chromStart','chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts']
    tempdf.reset_index(drop=True, inplace=True)
    for index, row in tempdf.iterrows():
        arr.append(file_df[file_df['name'] == row['ID']])
        found_row = file_df[file_df['name'] == row['ID']]
        # print(row['ID'])
        df.loc[df['ID'] == row['ID'], 'chr'] = found_row['chrom'].values[0].replace('chr', '')
        df.loc[df['ID'] == row['ID'], 'start'] = found_row['chromStart'].values[0]
        df.loc[df['ID'] == row['ID'], 'end'] = found_row['chromEnd'].values[0]

enhancers.bed
gc19_pc.3utr.bed
gc19_pc.5utr.bed
gc19_pc.cds.bed
gc19_pc.promCore.bed
gc19_pc.promDomain.bed
gc19_pc.ss.bed
lncrna.ncrna.bed
lncrna.promCore.bed
lncrna.promDomain.bed
lncrna.ss.bed
mirna.mat.bed
mirna.pre.bed
mirna.prom.bed
smallrna.ncrna.bed
supplementary_lncrna.ncrna.bed
supplementary_lncrna.prom.bed
supplementary_lncrna.ss.bed


In [39]:
file_df[file_df['reg_id'] == 'gc19_pc.promCore::gencode::CHMP4B::ENSG00000101421.3']

,region_type,mut_chr,mut_pos1,mut_pos2,mut_ref,mut_alt,mut_patient,mut_cc,mut_tag,reg_chr,reg_starts,reg_ends,reg_id,reg_frag_id
264519,gc19_pc.promCore.bed,chr20,32398974,32398974,A,G,DO49087,Panc-AdenoCA,AAG>G,chr20,32398909.0,32399274.0,gc19_pc.promCore::gencode::CHMP4B::ENSG0000010...,1
266069,gc19_pc.promCore.bed,chr20,32399061,32399061,T,C,DO35230,Panc-AdenoCA,GTG>C,chr20,32398909.0,32399274.0,gc19_pc.promCore::gencode::CHMP4B::ENSG0000010...,1
266105,gc19_pc.promCore.bed,chr20,32399051,32399051,T,G,DO218442,Stomach-AdenoCA,GTT>G,chr20,32398909.0,32399274.0,gc19_pc.promCore::gencode::CHMP4B::ENSG0000010...,1
266373,gc19_pc.promCore.bed,chr20,32398974,32398974,A,G,DO218223,Stomach-AdenoCA,AAG>G,chr20,32398909.0,32399274.0,gc19_pc.promCore::gencode::CHMP4B::ENSG0000010...,1
271428,gc19_pc.promCore.bed,chr20,32398939,32398939,A,T,DO50796,Liver-HCC,CAG>T,chr20,32398909.0,32399274.0,gc19_pc.promCore::gencode::CHMP4B::ENSG0000010...,1


In [54]:
arr = []
df['chr'] = ''
df['start'] = 0
df['end'] = 0
df['ref'] = 0
df['alt'] = 0
df['region_type'] = 0
df['found'] = 0
file_df = pd.read_csv('data/PCAWG_mutations_to_elements.icgc.public.txt', sep='\t')
for index, row in df.iterrows():
    found_row = file_df[file_df['reg_id'] == row['ID']]
    if len(found_row) > 1:
        print("FOUND", len(found_row))
    for i, r in found_row.iterrows():
        print(r[['mut_chr', 'mut_pos1', 'mut_pos2', 'mut_ref', 'mut_alt', 'region_type']])
    if len(found_row) > 0:
        df.loc[df['ID'] == row['ID'], 'chr'] = found_row['mut_chr'].values[0].replace('chr', '')
        df.loc[df['ID'] == row['ID'], 'start'] = found_row['mut_pos1'].values[0]
        df.loc[df['ID'] == row['ID'], 'end'] = found_row['mut_pos2'].values[0]
        df.loc[df['ID'] == row['ID'], 'ref'] = found_row['mut_ref'].values[0]
        df.loc[df['ID'] == row['ID'], 'alt'] = found_row['mut_alt'].values[0]
        df.loc[df['ID'] == row['ID'], 'region_type'] = found_row['region_type'].values[0]
        df.loc[df['ID'] == row['ID'], 'found'] = len(found_row)
    break

FOUND 5
mut_chr                       chr20
mut_pos1                   32398974
mut_pos2                   32398974
mut_ref                           A
mut_alt                           G
region_type    gc19_pc.promCore.bed
Name: 264519, dtype: object
mut_chr                       chr20
mut_pos1                   32399061
mut_pos2                   32399061
mut_ref                           T
mut_alt                           C
region_type    gc19_pc.promCore.bed
Name: 266069, dtype: object
mut_chr                       chr20
mut_pos1                   32399051
mut_pos2                   32399051
mut_ref                           T
mut_alt                           G
region_type    gc19_pc.promCore.bed
Name: 266105, dtype: object
mut_chr                       chr20
mut_pos1                   32398974
mut_pos2                   32398974
mut_ref                           A
mut_alt                           G
region_type    gc19_pc.promCore.bed
Name: 266373, dtype: object
mut_chr         

In [53]:
df#[df['start']>0]

,ID,target,tissue,gene,Brown_observed,max_method_contr,max_method_id,Pre-filter q-value,Post-filter q-value,Mutations,...,Mutated patients w. expression data,Wild-type patients with expression data,file_name,chr,start,end,ref,alt,region_type,found
0,gc19_pc.promCore::gencode::CHMP4B::ENSG0000010...,promCore,meta_Adenocarcinoma,CHMP4B,0.000194,0.373270,Fisher_observed,0.413950,1.0,8,...,4.0,765.0,gc19_pc.promCore.bed,None,None,None,None,None,None,5
1,gc19_pc.promCore::gencode::DYNLL1::ENSG0000008...,promCore,meta_Adenocarcinoma,DYNLL1,0.000248,0.371018,Fisher_observed,0.489554,1.0,15,...,8.0,761.0,gc19_pc.promCore.bed,None,None,None,None,None,None,9
2,gc19_pc.promCore::gencode::HES1::ENSG000001143...,promCore,meta_Adenocarcinoma,HES1,0.000109,0.378151,Fisher_observed,0.293439,1.0,9,...,6.0,763.0,gc19_pc.promCore.bed,None,None,None,None,None,None,3
3,gc19_pc.promCore::gencode::KB-1507C5.2::ENSG00...,promCore,meta_Adenocarcinoma,KB-1507C5.2,0.000175,0.374191,Fisher_observed,0.388602,1.0,9,...,2.0,767.0,gc19_pc.promCore.bed,None,None,None,None,None,None,6
4,gc19_pc.promCore::gencode::LEPROTL1::ENSG00000...,promCore,meta_Adenocarcinoma,LEPROTL1,0.000151,0.375440,Fisher_observed,0.361455,1.0,9,...,5.0,764.0,gc19_pc.promCore.bed,None,None,None,None,None,None,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203,smallrna.ncrna::trnascanse::Asn.GTT::id.trna.578,smallrna,Thy-AdenoCa,Asn.GTT,0.000068,0.359477,Fisher_observed,0.247733,1.0,2,...,NaN,NaN,smallrna.ncrna.bed,[],[],[],[],[],[],0
1204,smallrna.ncrna::trnascanse::Asp.GTC::id.trna.253,smallrna,Thy-AdenoCa,Asp.GTC,0.000238,0.348613,Fisher_observed,0.442825,1.0,1,...,NaN,NaN,smallrna.ncrna.bed,[],[],[],[],[],[],0
1205,smallrna.ncrna::trnascanse::Cys.GCA::id.trna.394,smallrna,Thy-AdenoCa,Cys.GCA,0.000197,0.350368,Fisher_observed,0.414456,1.0,1,...,NaN,NaN,smallrna.ncrna.bed,[],[],[],[],[],[],0
1206,smallrna.ncrna::gencode::SNORD115-17::ENSG0000...,smallrna,Uterus-AdenoCa,SNORD115-17,0.000066,0.380399,Fisher_observed,0.246445,1.0,2,...,NaN,NaN,smallrna.ncrna.bed,[],[],[],[],[],[],0


In [ ]:
df

In [14]:
df

,ID,target,tissue,gene,Brown_observed,max_method_contr,max_method_id,Pre-filter q-value,Post-filter q-value,Mutations,...,Overlap,Final filter judgement,Expression_p-value,Expression_Fold Difference,Mutated patients w. expression data,Wild-type patients with expression data,file_name,chr,start,end
0,gc19_pc.promCore::gencode::CHMP4B::ENSG0000010...,promCore,meta_Adenocarcinoma,CHMP4B,0.000194,0.373270,Fisher_observed,0.413950,1.0,8,...,True,True,0.702076,1.216133,4.0,765.0,gc19_pc.promCore.bed,20,32398909,32399274
1,gc19_pc.promCore::gencode::DYNLL1::ENSG0000008...,promCore,meta_Adenocarcinoma,DYNLL1,0.000248,0.371018,Fisher_observed,0.489554,1.0,15,...,True,True,0.694475,0.991591,8.0,761.0,gc19_pc.promCore.bed,12,120907452,120934115
2,gc19_pc.promCore::gencode::HES1::ENSG000001143...,promCore,meta_Adenocarcinoma,HES1,0.000109,0.378151,Fisher_observed,0.293439,1.0,9,...,False,False,0.242469,0.546071,6.0,763.0,gc19_pc.promCore.bed,3,193853733,193854133
3,gc19_pc.promCore::gencode::KB-1507C5.2::ENSG00...,promCore,meta_Adenocarcinoma,KB-1507C5.2,0.000175,0.374191,Fisher_observed,0.388602,1.0,9,...,True,True,0.231362,1.838830,2.0,767.0,gc19_pc.promCore.bed,8,103876327,103876578
4,gc19_pc.promCore::gencode::LEPROTL1::ENSG00000...,promCore,meta_Adenocarcinoma,LEPROTL1,0.000151,0.375440,Fisher_observed,0.361455,1.0,9,...,True,False,0.050680,1.203716,5.0,764.0,gc19_pc.promCore.bed,8,29952713,29959655
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203,smallrna.ncrna::trnascanse::Asn.GTT::id.trna.578,smallrna,Thy-AdenoCa,Asn.GTT,0.000068,0.359477,Fisher_observed,0.247733,1.0,2,...,True,False,NaN,NaN,NaN,NaN,smallrna.ncrna.bed,17,36908033,36908107
1204,smallrna.ncrna::trnascanse::Asp.GTC::id.trna.253,smallrna,Thy-AdenoCa,Asp.GTC,0.000238,0.348613,Fisher_observed,0.442825,1.0,1,...,True,False,NaN,NaN,NaN,NaN,smallrna.ncrna.bed,6,27447452,27447524
1205,smallrna.ncrna::trnascanse::Cys.GCA::id.trna.394,smallrna,Thy-AdenoCa,Cys.GCA,0.000197,0.350368,Fisher_observed,0.414456,1.0,1,...,True,False,NaN,NaN,NaN,NaN,smallrna.ncrna.bed,7,149295045,149295117
1206,smallrna.ncrna::gencode::SNORD115-17::ENSG0000...,smallrna,Uterus-AdenoCa,SNORD115-17,0.000066,0.380399,Fisher_observed,0.246445,1.0,2,...,True,False,NaN,NaN,NaN,NaN,smallrna.ncrna.bed,15,25446469,25446551


In [5]:
raw_data = pd.read_csv('data/dataset_uncensored.csv')
raw_data['start'] = raw_data['start'].astype(int)
raw_data['end'] = raw_data['end'].astype(int)

In [ ]:
overlaps = check_overlaps(df, raw_data)

In [10]:
overlaps['target'].value_counts()

promCore           28
lncRNA_promCore    16
5UTR               15
enhancer            5
lncRNA_ncrna        4
3UTR                3
Name: target, dtype: int64

In [19]:
arr_ids = []
arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/indel'))
arr_ids  = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/snv_mnv'))
# arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/graylist'))
arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_tcga.controlled/final_consensus_12oct/tcga/indel'))
arr_ids  = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_tcga.controlled/final_consensus_12oct/tcga/snv_mnv'))
# arr_ids = arr_ids + (os.listdir('data/Rheinbay 2020/final_consensus_snv_indel_tcga.controlled/final_consensus_12oct/tcga/graylist'))
arr_ids = [s for s in arr_ids if '.tbi' not in s]
arr_ids_names = [s[:s.find(".")] for s in arr_ids]

In [ ]:
def read_vcf_gzip(filename):
    with gzip.open(filename, 'r') as f:
        lines = [l for l in f if not l.startswith(b'##')]
    df = pd.read_csv(
        io.StringIO(b''.join(lines).decode()),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
                'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})
    return df
    
read_vcf_gzip('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/indel/0009b464-b376-4fbc-8a56-da538269a02f.consensus.20161006.somatic.indel.vcf.gz')

In [7]:
all_files = [val for sublist in [[os.path.join(i[0], j) for j in i[2]] for i in os.walk('D:\Masters Thesis\GitHub Repo\data\gene-coding-and-non-coding-elements')] for val in sublist]
all_files = [s for s in all_files if 'allCombined' not in s]
all_files = [s for s in all_files if 'mapFiles' not in s]
all_files

['D:\\Masters Thesis\\GitHub Repo\\data\\gene-coding-and-non-coding-elements\\enhancers.bed',
 'D:\\Masters Thesis\\GitHub Repo\\data\\gene-coding-and-non-coding-elements\\gc19_pc.3utr.bed',
 'D:\\Masters Thesis\\GitHub Repo\\data\\gene-coding-and-non-coding-elements\\gc19_pc.5utr.bed',
 'D:\\Masters Thesis\\GitHub Repo\\data\\gene-coding-and-non-coding-elements\\gc19_pc.cds.bed',
 'D:\\Masters Thesis\\GitHub Repo\\data\\gene-coding-and-non-coding-elements\\gc19_pc.promCore.bed',
 'D:\\Masters Thesis\\GitHub Repo\\data\\gene-coding-and-non-coding-elements\\gc19_pc.promDomain.bed',
 'D:\\Masters Thesis\\GitHub Repo\\data\\gene-coding-and-non-coding-elements\\gc19_pc.ss.bed',
 'D:\\Masters Thesis\\GitHub Repo\\data\\gene-coding-and-non-coding-elements\\lncrna.ncrna.bed',
 'D:\\Masters Thesis\\GitHub Repo\\data\\gene-coding-and-non-coding-elements\\lncrna.promCore.bed',
 'D:\\Masters Thesis\\GitHub Repo\\data\\gene-coding-and-non-coding-elements\\lncrna.promDomain.bed',
 'D:\\Masters Thes

In [8]:
import re
all_df = pd.DataFrame()
all_df['element'] = ''
for file in all_files:
    temp_df = pd.read_csv(file, sep='\t', header=None)
    result = re.search('elements\\\\(.*).bed', file)
    temp_df['element'] = str(result.group(1))
    all_df = pd.concat([all_df, temp_df])
all_df.columns = ['element', 'chrom', 'chromStart','chromEnd', 'name', 'score', 'strand', 'thickStart', 'thickEnd', 'itemRgb', 'blockCount', 'blockSizes', 'blockStarts']
all_df['chromStart'] = all_df['chromStart'].astype(int)
all_df['chromEnd'] = all_df['chromEnd'].astype(int)
all_df['score'] = all_df['score'].astype(int)
all_df['chrom'] = all_df['chrom'].apply(lambda x: x.replace('chr', ''))

all_df.rename({'chrom': 'chr', 'chromStart': 'start', 'chromEnd': 'end'}, inplace=True, axis = 1)

In [9]:
overlaps = check_overlaps_and_annotate(raw_data, all_df, 'element')

Found overlaps


In [13]:
overlaps[~overlaps['annotation'].isna()]

,id,chr,start,end,ref,alt,driver,data_source,found,annotation
0,mut0,3,101578254,101578255,CAGTT,C,1,ICGC,1,gc19_pc.3utr
1,mut1,3,101578285,101578286,GACCATTTGCCTT,G,1,ICGC,1,gc19_pc.3utr
2,mut2,3,101578250,101578251,CTG,C,1,ICGC,1,gc19_pc.3utr
3,mut3,17,48940015,48940016,CTAAAT,C,1,ICGC,1,gc19_pc.3utr
4,mut4,17,48939986,48939987,AT,A,1,ICGC,1,gc19_pc.3utr
...,...,...,...,...,...,...,...,...,...,...
1081,mut1081,4,95496939,95496940,C,T,0,COSMIC,1,gc19_pc.ss
1082,mut1082,6,53026396,53026397,C,T,0,COSMIC,1,supplementary_lncrna.prom
1083,mut1083,2,220077160,220077161,G,T,0,COSMIC,1,gc19_pc.ss
1084,mut1084,16,31471209,31471210,C,T,0,COSMIC,1,gc19_pc.ss


In [112]:
overlaps1 = check_overlaps(all_df, raw_data)

Found overlaps


In [114]:
overlaps1['element'].value_counts()

supplementary_lncrna.ncrna    184
supplementary_lncrna.prom     170
supplementary_lncrna.ss        90
enhancers                       5
gc19_pc.promCore                3
gc19_pc.promDomain              3
gc19_pc.ss                      3
gc19_pc.3utr                    3
gc19_pc.cds                     3
gc19_pc.5utr                    3
lncrna.ncrna                    2
lncrna.promCore                 2
lncrna.ss                       2
mirna.mat                       2
mirna.pre                       2
mirna.prom                      2
smallrna.ncrna                  2
lncrna.promDomain               2
Name: element, dtype: int64

In [3]:
def row_count(input):
    with open(input) as f:
        for i, l in enumerate(f):
            pass
    return i

row_count('data/Rheinbay 2020/final_consensus_passonly.snv_mnv_indel.icgc.controlled.maf')

23159591

In [37]:
arr_ids.index('6f25ca85-4b4a-420f-b0cc-5a4f65a97da3')

1541

In [25]:
np.array('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/indel')

array('data/Rheinbay 2020/final_consensus_snv_indel_passonly_icgc.public/indel',
      dtype='<U71')